In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.io_utils import *
from src.metrics import *

df = read_data("data/ecommerce_sales.csv")

df_clean = clean_data(df)

print("---" * 50)
print()
print("Here is a sample of our data:")
print()
print(df_clean.sample(3))
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

Here is a sample of our data:

     order_id       date       city  category   price  units  revenue
734    100734 2024-01-25    Uppsala  Clothing  412.16      4  1648.64
397    100397 2024-04-20  Stockholm  Clothing  381.31      3  1143.93
303    100303 2024-06-06   Västerås    Beauty  247.93      4   991.72

------------------------------------------------------------------------------------------------------------------------------------------------------


## Total revenue and units sold

In [2]:
print("---" * 50)
print()
print(f"In 2024 we sold a total of {total_units(df_clean)} units,\n"
      f"for a total revenue of {total_revenue(df_clean)} sek.")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

In 2024 we sold a total of 7463 units,
for a total revenue of 3502767.23 sek.

------------------------------------------------------------------------------------------------------------------------------------------------------


## Average Order

In [3]:
print("---" * 50)
print()
print(f"Th average order value is {average_order(df_clean)} sek.")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

Th average order value is 1401.11 sek.

------------------------------------------------------------------------------------------------------------------------------------------------------


## Revenue per category

In [4]:
print("---" * 50)
print()
print(f"The top 3 categories per revenue are:\n"
      f"{revenue_per_category(df_clean).head(3)}.")
print("---INSERT PLOT")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

The top 3 categories per revenue are:
      category  tot_rev_cat
0  Electronics   1364228.07
1       Sports    734653.42
2     Clothing    586744.91.
---INSERT PLOT

------------------------------------------------------------------------------------------------------------------------------------------------------


## Revenue per city

In [10]:
print("---" * 50)
print()
print(f"The top 3 cities per revenue are:\n"
      f"{revenue_per_city(df_clean).head(3)}.")
print("---INSERT PLOT")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

The top 3 cities per revenue are:
        city  tot_rev_city
0  Stockholm    1227201.94
1   Göteborg     859019.65
2      Malmö     567926.32.
---INSERT PLOT

------------------------------------------------------------------------------------------------------------------------------------------------------
